In [1]:
from langchain_community.llms import Ollama
llm= Ollama(model="llama2")

In [2]:
# import os 
# from dotenv import load_dotenv

# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)

In [3]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI()

In [2]:
llm.invoke("how can langsmith help with testing?")

: 

In [2]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [3]:
chain = prompt | llm

In [6]:
chain.invoke({"input":"how can langsmith help with testing?"})

: 

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [5]:
chain = prompt | llm | output_parser

In [8]:
chain.invoke({"input":"how can langsmith help with testing?"})

'Langsmith can help with testing in various ways, including:\n\n1. Automated Testing: Langsmith can be used to create automated tests for your software applications. You can write test cases in Langsmith that mimic user interactions with your application, helping to ensure its functionality and performance.\n\n2. Load Testing: Langsmith can simulate multiple users interacting with your application simultaneously, allowing you to test how well your application handles heavy traffic and load.\n\n3. Integration Testing: Langsmith can be used to create integration tests to verify that different components of your application work together correctly.\n\n4. Regression Testing: Langsmith can help you automate regression tests to ensure that new changes or updates to your application do not introduce any new bugs or issues.\n\n5. Performance Testing: Langsmith can be used to conduct performance tests to measure how well your application performs under various conditions, helping you identify b

# Retrieval chain

In [7]:
from langchain_community.document_loaders import WebBaseLoader
# loader = WebBaseLoader("https://docs.smith.lchangchain.com")
# loader = WebBaseLoader("https://edisciplinas.usp.br/pluginfile.php/5922722/mod_resource/content/1/2013%20-%20Book%20-%20Bass%20%20Kazman-Software%20Architecture%20in%20Practice%20%281%29.pdf")
loader = WebBaseLoader("https://drive.google.com/file/d/1grZyR7sXYPY0eiSqh_Yr5-99xPZ7snL_/view?usp=drive_link")

docs = loader.load()

In [4]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()

In [8]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [8]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()

chunk_size = 100
for i in range(0, len(docs), chunk_size):
    # for testing
    if i ==0: 
        chunk = docs[i:i+chunk_size]
        documnets = text_splitter.split_documents(chunk)
        vector = FAISS.from_documents(documnets, embeddings)
    
    
    

: 

In [9]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based on the provided context:
    <context>
    {context}
    <context>
    
    Question: {input}""")

document_chain = create_stuff_documents_chain(llm,prompt)

In [ ]:
# pass documents directly
from langchain_core.documents import Document

document_chain.invoke({
    "input":"how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})


'Langsmith can help with testing by allowing users to visualize test results.'

In [11]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [12]:
response = retrieval_chain.invoke({"input":"What is a lease?"})
print(response["answer"])


A lease is a legal agreement between a lessor (the owner of the property) and a lessee (the individual or entity renting the property) that grants the right to use the property for a specific period of time in exchange for regular payments, known as rent. The lease outlines the terms and conditions of the rental agreement, including the length of the lease, the rent amount, and any specific restrictions or obligations on the use of the property. Leases are commonly used for residential and commercial properties, and can also be used for other types of assets, such as equipment or vehicles.


# Conversation retrieval chain

In [14]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    ("user","Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retrieval_chain = create_history_aware_retriever(llm,retriever,prompt)

In [19]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input":"Explain how different lease types work"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Explain how different lease types work',
 'context': [Document(page_content='Chapter 3  -  Leases.pdf - Google Driveඇතුළු වන්නපූර්ණය කරමින්…', metadata={'source': 'https://drive.google.com/file/d/1grZyR7sXYPY0eiSqh_Yr5-99xPZ7snL_/view?usp=drive_link', 'title': 'Chapter 3  -  Leases.pdf - Google Drive', 'language': 'No language found.'})],
 'answer': "Of course, I'd be happy to help you understand the different types of leases and how they work!\n\nA lease is a legal agreement between a property owner (lessor) and a tenant (lessee) that grants the right to occupy and use a property for a specific period of time in exchange for rent. There are several types of leases, including:\n\n1. Fixed-term lease: This is the most common type of lease, where the lease agreement specifies the start and end dates, and the rent amount. The lease will automatically renew at th

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Answer the user's questions baseed on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}")
])

document_chain = create_stuff_documents_chain(llm,prompt)
retrieval_chain = create_retrieval_chain(retrieval_chain,document_chain)